# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [21]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, bias=False)
        self.bn1 = nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=True)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, bias=False)
        self.bn2 = nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=True)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, bias=False)
        self.bn3 = nn.BatchNorm2d(128, eps=1e-5, momentum=0.1, affine=True)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, bias=False)
        self.bn4 = nn.BatchNorm2d(256, eps=1e-5, momentum=0.1, affine=True)
        
        self.relu = nn.ReLU(inplace=True)
        self.avgpool_1 = nn.AvgPool2d(kernel_size=3,stride=1, padding=1)
        self.avgpool_2 = nn.AvgPool2d(kernel_size=3,stride=2, padding=1)
        self.avgpool_3 = nn.AvgPool2d(kernel_size=3, stride=1)
        
        self.linear_1 = nn.Linear(256, 128)
        self.linear_2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.avgpool_1(self.relu(self.bn1(self.conv1(x))))    # 32 * 26 * 26
        x = self.avgpool_2(self.relu(self.bn2(self.conv2(x))))    # 64 * 12 * 12
        x = self.avgpool_2(self.relu(self.bn3(self.conv3(x))))    # 128 * 5 * 5
        x = self.avgpool_3(self.relu(self.bn4(self.conv4(x))))      # 256 * 1 * 1
        x = x.view(x.size(0), -1)
        x = self.relu(self.linear_1(x))
        x = self.linear_2(x)
        return x

    import torch.backends.cudnn as cudnn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SimpleNet().to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 4, gamma=0.1)

Next, we can start to train and evaluate!

In [19]:
# train and evaluate
best_acc = 0.0
for epoch in range(NUM_EPOCHS):
    model.train()
    correct_train = 0
    total_train = 0
    correct_test = 0
    total_test = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predict = output.max(1)
        total_train += labels.size(0)
        correct_train += predict.eq(labels).sum().item()
    
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predict = output.max(1)
            total_test += labels.size(0)
            correct_test += predict.eq(labels).sum().item()
    
    train_acc = correct_train / total_train
    test_acc = correct_test / total_test
    print(f'##### Epoch {epoch} | train acc: {train_acc} | test acc: {test_acc} #####')
    scheduler.step() 
    best_acc = max(best_acc, test_acc)

  1%|▌                                                                                 | 3/468 [00:00<00:17, 26.62it/s]

##### Epoch 0 | train acc: 0.9696848290598291 | test acc: 0.9770633012820513 #####


  1%|▌                                                                                 | 3/468 [00:00<00:17, 26.13it/s]

##### Epoch 1 | train acc: 0.9879139957264957 | test acc: 0.9892828525641025 #####


  1%|▌                                                                                 | 3/468 [00:00<00:16, 27.43it/s]

...

##### Epoch 7 | train acc: 0.9976462339743589 | test acc: 0.995292467948718 #####


  1%|▌                                                                                 | 3/468 [00:00<00:18, 24.92it/s]

##### Epoch 8 | train acc: 0.9978131677350427 | test acc: 0.9955929487179487 #####


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 50.20it/s]

##### Epoch 9 | train acc: 0.9982805822649573 | test acc: 0.995292467948718 #####


In [20]:
print(f'##### Epoch {epoch} | train acc: {train_acc} | test acc: {best_acc} #####')

##### Epoch 9 | train acc: 0.9982805822649573 | test acc: 0.9955929487179487 #####


#### Q5:
Please print the training and testing accuracy.